# IBM Capstone Project

## Business opportunities near train stations

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



## Import data

In [2]:
df1 = pd.read_csv('train_stations_europe.csv')

In [3]:
df1.head()

,id,name,name_norm,uic,latitude,longitude,parent_station_id,country,time_zone,is_city,is_main_station,is_airport
0,1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,NaN,44.081790,6.001625,NaN,FR,Europe/Paris,True,False,False
1,2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,8775123.0,44.061565,5.997373,1.0,FR,Europe/Paris,False,True,False
2,3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,8775122.0,44.063863,6.011248,1.0,FR,Europe/Paris,False,False,False
3,4,Digne-les-Bains,Digne-les-Bains,9921024.0,44.350000,6.350000,NaN,FR,Europe/Paris,True,False,False
4,6,Digne-les-Bains,Digne-les-Bains,8775149.0,44.088710,6.222982,4.0,FR,Europe/Paris,False,True,False


id: Numeric internal unique identifier. Primary key.

name: Name of the station as it is locally known. These names include accents and other special characters.

name_norm: Normalised version of name; transformed into [A-Za-z] character space (aka 'Latin-ASCII') to replace special characters with their standard-Latin counterparts (e.g. è become e, ü becomes u).

uic: The UIC code of the station. UIC is the International Union of Railways, "an international rail transport industry body". About 1/3 of all stations have no UIC code in this dataset.

longitude & latitude: Station coordinates. About 5% of all stations have no coordinates in this dataset.

parent_station_id: A station can belong to a meta station whose id is this value, i.e. Paris Gare d’Austerlitz (id = 4921) belongs to the meta-station Paris (id = 4916). About 92% of rows have NA entries.

country: Country codes in ISO 3166-1 alpha-2 format (2 digits).

time_zone: Continent/Country ISO codes. Those appear to be equivalent to Olson names (e.g. "Europe/Berlin").

is_city: Marked as "unreliable" in the source dataset. Might be worth investigating what exactly that means.

is_main_station: Marked as "unreliable" in the source dataset. Might be worth investigating what exactly that means.

## Data cleansing

In [4]:
country = df1.set_index('id')

In [5]:
country.head()

,name,name_norm,uic,latitude,longitude,parent_station_id,country,time_zone,is_city,is_main_station,is_airport
id,,,,,,,,,,,
1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,NaN,44.081790,6.001625,NaN,FR,Europe/Paris,True,False,False
2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,8775123.0,44.061565,5.997373,1.0,FR,Europe/Paris,False,True,False
3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,8775122.0,44.063863,6.011248,1.0,FR,Europe/Paris,False,False,False
4,Digne-les-Bains,Digne-les-Bains,9921024.0,44.350000,6.350000,NaN,FR,Europe/Paris,True,False,False
6,Digne-les-Bains,Digne-les-Bains,8775149.0,44.088710,6.222982,4.0,FR,Europe/Paris,False,True,False


In [6]:
country = country.drop(columns = ['uic', 'parent_station_id', 'time_zone', 'is_airport'])
country.head()

,name,name_norm,latitude,longitude,country,is_city,is_main_station
id,,,,,,,
1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,44.081790,6.001625,FR,True,False
2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,44.061565,5.997373,FR,False,True
3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,44.063863,6.011248,FR,False,False
4,Digne-les-Bains,Digne-les-Bains,44.350000,6.350000,FR,True,False
6,Digne-les-Bains,Digne-les-Bains,44.088710,6.222982,FR,False,True


In [26]:
country['is_main_station'].value_counts()

False    56173
True       817
Name: is_main_station, dtype: int64

In [27]:
country['is_city'].value_counts()

False    48467
True      8523
Name: is_city, dtype: int64

In [28]:
ms_df = country.loc[country['is_main_station'] == True]
ms_df.head()

,name,name_norm,latitude,longitude,country,is_city,is_main_station
id,,,,,,,
2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,44.061565,5.997373,FR,False,True
6,Digne-les-Bains,Digne-les-Bains,44.088710,6.222982,FR,False,True
9,Cagnes-sur-Mer,Cagnes-sur-Mer,43.658130,7.148640,FR,False,True
11,Menton,Menton,43.774444,7.493300,FR,False,True
18,Vievola,Vievola,44.112556,7.562798,FR,False,True


In [29]:
ms_df.shape

(817, 7)

In [30]:
Europe_ms_df = ms_df.drop(columns = ['is_city', 'is_main_station', 'country', 'name_norm'])
Europe_ms_df.head()

,name,latitude,longitude
id,,,
2,Château-Arnoux—St-Auban,44.061565,5.997373
6,Digne-les-Bains,44.088710,6.222982
9,Cagnes-sur-Mer,43.658130,7.148640
11,Menton,43.774444,7.493300
18,Vievola,44.112556,7.562798


In [31]:
Europe_ms_df.reset_index(drop=True, inplace=True)
Europe_ms_df.head()

,name,latitude,longitude
0,Château-Arnoux—St-Auban,44.061565,5.997373
1,Digne-les-Bains,44.088710,6.222982
2,Cagnes-sur-Mer,43.658130,7.148640
3,Menton,43.774444,7.493300
4,Vievola,44.112556,7.562798


In [37]:
Europe_ms_df.isnull().sum()

name         0
latitude     1
longitude    1
dtype: int64

In [38]:
europe_df = Europe_ms_df.dropna(how='any',axis=0) 

In [39]:
europe_df.isnull().sum()

name         0
latitude     0
longitude    0
dtype: int64

## Foursquare credentials

In [20]:
CLIENT_ID = 'OQURYFVKSGDBTVGE1U44MPO1LO4PKFDGCVRYZXOQX5LDCULO' # your Foursquare ID
CLIENT_SECRET = '2W31B1UE1OY3OTNRJZR52PZS0JA4Z3IXJX115SOTPDC2QPYP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQURYFVKSGDBTVGE1U44MPO1LO4PKFDGCVRYZXOQX5LDCULO
CLIENT_SECRET:2W31B1UE1OY3OTNRJZR52PZS0JA4Z3IXJX115SOTPDC2QPYP


In [32]:
address = 'Menton'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7753495 7.5029213


## Visualizing major stations in Europe

In [44]:
europe_map = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, label in zip(europe_df['latitude'], europe_df['longitude'], europe_df['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.4,
        parse_html=False).add_to(europe_map)  
    
europe_map

## Creating list of venues and merging it to europe dataframe

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius=800&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Train station', 
                  'Train station Latitude', 
                  'Train station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
europe_venues = getNearbyVenues(names = europe_df['name'],
                                latitudes = europe_df['latitude'],
                                longitudes = europe_df['longitude'])

Château-Arnoux—St-Auban
Digne-les-Bains
Cagnes-sur-Mer
Menton
Vievola
Decazeville Centre
St-Sever Calvados
Vire
Marans
Uzerche
Dijon Ville
Pontrieux
Besançon Viotte
Colomiers
Labège Innopole
Albas
Lunel
Moirans
Dole Ville
Rezé—Pont-Rousseau
Angers St-Laud
Saumur
Reims Ville
Louverné
Mayenne
Arnage
Carnac
St-Pierre-Quiberon
Metz
Forbach
Sarreguemines
Lille-Flandres
Tourcoing
Nomain
Croix—Wasquehal
Jeumont
Bellegarde-sur-Valserine
Calais Ville
Boulogne-sur-Mer Ville
Pau
Strasbourg
Mertzwiller
Colmar
St-Louis
Thann
Aspach-le-Haut
Avignon Centre
Eu
Harfleur
Montivilliers
Dieppe
Péronne Centre
Albi Ville
Orléans Centre
Fontenay-le-Comte
St-Hilaire-de-Riez
Tours
Limoges Bénédictins
Argenton-sur-Creuse
Roye
Versailles-Chantiers
Auxon Église
Lexy
Romilly-sur-Seine
Nogent-sur-Seine
Vendeuvre
Bar-sur-Aube
Bologne
Vignory
Gudmont
Mirecourt
Bagnoles-de-l’Orne Office de Tourisme
Carignan
Margut Fromy
Vitry-le-François
St-Dizier
Fronville
Eurville
Jussey
Barberey Route de Méry
Kuntzig
Hombourg-Budan

Disentis/Muster
Sargans
Visp
Lauterbrunnen
Interlaken Ost
Plzeň hl.n.
Herning St.
Nykøbing Falster St.
Tirano
Como San Giovanni
Leiden Lammenschans
Ronneby station
Ancona stazione
Catania Centrale
Ciampino
Bari Centrale
Benevento stazione
Catanzaro (Germaneto)
Livorno Centrale
Oliveri-Tindari
Pescara Centrale
Palermo Centrale
Montecatini Centro
Lamezia Terme Centrale
Prato Centrale
Modugno città
Messina Centrale
Gorizia Centrale
Igea Marina
Pisa Centrale
Brolo-Ficarra
Cervia-Milano Marittima
Malpensa Aeroporto T1
Reggio Emilia stazione
Treviso Centrale
Trieste Centrale
Agrigento Centrale
Civitavecchia stazione
La Spezia Centrale
Cattolica-San Giovanni-Gabicce
Orbetello-Monte Argentario
Agropoli-Castellabate
Potenza Centrale
Todi Ponte Rio
Borgo Valsugana Est
Nardò Città
Luxembourg Gare Centrale (Quai 13)
Zagreb Autobusni Kolodvor
Wien Hbf
Zaragoza Delicias
Bastogne-Sud
Aranguren Apeadero
Cellers-Llimiana
Cuenca Estación
Guadalajara Estación
Lantueno-Santiurde
Lorca-San Diego
Sabadell C

In [46]:
europe_venues.head()

,Train station,Train station Latitude,Train station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Château-Arnoux—St-Auban,44.061565,5.997373,Gare SNCF de Château-Arnoux-Saint-Auban,44.061557,5.997516,Train Station
1,Château-Arnoux—St-Auban,44.061565,5.997373,Aérodrome de Saint-Auban (LFMX),44.059547,5.987779,Airport
2,Digne-les-Bains,44.088710,6.222982,Gare de Digne-les-Bains,44.088760,6.222702,Train Station
3,Digne-les-Bains,44.088710,6.222982,Jardin public,44.085594,6.225947,Park
4,Digne-les-Bains,44.088710,6.222982,LEDDA Modding Design,44.086553,6.229486,IT Services


In [47]:
europe_venues.drop(europe_venues[europe_venues['Venue Category'] == 'Train Station'].index, inplace = True)
europe_venues.head()

,Train station,Train station Latitude,Train station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Château-Arnoux—St-Auban,44.061565,5.997373,Aérodrome de Saint-Auban (LFMX),44.059547,5.987779,Airport
3,Digne-les-Bains,44.088710,6.222982,Jardin public,44.085594,6.225947,Park
4,Digne-les-Bains,44.088710,6.222982,LEDDA Modding Design,44.086553,6.229486,IT Services
5,Digne-les-Bains,44.088710,6.222982,Cinéma Ciné-Toiles,44.090920,6.231572,Multiplex
6,Digne-les-Bains,44.088710,6.222982,Brasserie De L'Etoile,44.091053,6.231764,French Restaurant


In [48]:
europe_venues.drop(europe_venues[europe_venues['Venue Category'] == 'Airport'].index, inplace = True)
europe_venues.head()

,Train station,Train station Latitude,Train station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Digne-les-Bains,44.08871,6.222982,Jardin public,44.085594,6.225947,Park
4,Digne-les-Bains,44.08871,6.222982,LEDDA Modding Design,44.086553,6.229486,IT Services
5,Digne-les-Bains,44.08871,6.222982,Cinéma Ciné-Toiles,44.090920,6.231572,Multiplex
6,Digne-les-Bains,44.08871,6.222982,Brasserie De L'Etoile,44.091053,6.231764,French Restaurant
7,Digne-les-Bains,44.08871,6.222982,L'ESCALE,44.090924,6.231964,Restaurant


In [49]:
europe_venues.reset_index(drop=True, inplace=True)
europe_venues.head()

,Train station,Train station Latitude,Train station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Digne-les-Bains,44.08871,6.222982,Jardin public,44.085594,6.225947,Park
1,Digne-les-Bains,44.08871,6.222982,LEDDA Modding Design,44.086553,6.229486,IT Services
2,Digne-les-Bains,44.08871,6.222982,Cinéma Ciné-Toiles,44.090920,6.231572,Multiplex
3,Digne-les-Bains,44.08871,6.222982,Brasserie De L'Etoile,44.091053,6.231764,French Restaurant
4,Digne-les-Bains,44.08871,6.222982,L'ESCALE,44.090924,6.231964,Restaurant


## Europe venues count

In [50]:
europe_venues.groupby('Train station').count()

,Train station Latitude,Train station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Train station,,,,,,
A Coruña-San Cristovo,54,54,54,54,54,54
Aachen Hbf,83,83,83,83,83,83
Aarau Bahnhof,31,31,31,31,31,31
Affoltern am Albis,9,9,9,9,9,9
Agrigento Centrale,25,25,25,25,25,25
...,...,...,...,...,...,...
Égletons,2,2,2,2,2,2
Évian-les-Bains,7,7,7,7,7,7
Évry,4,4,4,4,4,4


In [51]:
europe_onehot = pd.get_dummies(europe_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
europe_onehot['Train station'] = europe_venues['Train station'] 

# move neighborhood column to the first column
fixed_columns = [europe_onehot.columns[-1]] + list(europe_onehot.columns[:-1])
europe_onehot = europe_onehot[fixed_columns]

europe_onehot.head()

,Train station,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Digne-les-Bains,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Digne-les-Bains,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Digne-les-Bains,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Digne-les-Bains,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Digne-les-Bains,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
europe_onehot.shape
europe_grouped = europe_onehot.groupby('Train station').mean().reset_index()
europe_grouped

,Train station,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,A Coruña-San Cristovo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aachen Hbf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aarau Bahnhof,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Affoltern am Albis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Agrigento Centrale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,Égletons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787,Évian-les-Bains,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
788,Évry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
789,Ústí nad Labem hl.n.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Most common venues

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Train station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
europe_venues_sorted = pd.DataFrame(columns=columns)
europe_venues_sorted['Train station'] = europe_grouped['Train station']

for ind in np.arange(europe_grouped.shape[0]):
    europe_venues_sorted.iloc[ind, 1:] = return_most_common_venues(europe_grouped.iloc[ind, :], num_top_venues)

europe_venues_sorted.head()

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A Coruña-San Cristovo,Plaza,Spanish Restaurant,Coffee Shop,Tapas Restaurant,Restaurant,Pizza Place,Hotel,Brewery,Café,Seafood Restaurant
1,Aachen Hbf,Supermarket,Italian Restaurant,Bar,Bakery,German Restaurant,Drugstore,Café,Vietnamese Restaurant,Thai Restaurant,Movie Theater
2,Aarau Bahnhof,Café,Bar,Restaurant,Italian Restaurant,Fast Food Restaurant,Pizza Place,Supermarket,Irish Pub,Gym,Grocery Store
3,Affoltern am Albis,Grocery Store,Italian Restaurant,Gas Station,Coffee Shop,Eastern European Restaurant,Tapas Restaurant,Fast Food Restaurant,Department Store,Film Studio,Filipino Restaurant
4,Agrigento Centrale,Italian Restaurant,Sicilian Restaurant,Café,Restaurant,Pub,Park,Bakery,Food,Cocktail Bar,Ice Cream Shop


## Clusters creation

In [55]:
# set number of clusters
kclusters = 7

europe_grouped_clustering = europe_grouped.drop('Train station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(europe_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 0, 4, 3, 3, 1, 5, 3, 3])

In [59]:
europe_df = europe_df.rename(columns = {'name':'Train station'})
europe_df.head()

,Train station,latitude,longitude
0,Château-Arnoux—St-Auban,44.061565,5.997373
1,Digne-les-Bains,44.088710,6.222982
2,Cagnes-sur-Mer,43.658130,7.148640
3,Menton,43.774444,7.493300
4,Vievola,44.112556,7.562798


In [63]:
europe_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

europe_merged = europe_df

europe_merged = europe_merged.join(europe_venues_sorted.set_index('Train station'), on='Train station',how='right')

europe_merged.head()

,Train station,latitude,longitude,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Digne-les-Bains,44.088710,6.222982,3,3,Park,Multiplex,French Restaurant,Restaurant,IT Services,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
2,Cagnes-sur-Mer,43.658130,7.148640,1,1,Supermarket,Bakery,Diner,Pharmacy,Falafel Restaurant,Plaza,Resort,Food Truck,Filipino Restaurant,Franconian Restaurant
3,Menton,43.774444,7.493300,2,2,Hotel,Italian Restaurant,Beach,Café,Mediterranean Restaurant,Bakery,Vegetarian / Vegan Restaurant,Pedestrian Plaza,Resort,Ice Cream Shop
5,Decazeville Centre,44.561434,2.255896,3,3,Supermarket,Medical Supply Store,Toy / Game Store,Pizza Place,Mobile Phone Shop,Miscellaneous Shop,Fast Food Restaurant,Photography Studio,Zoo Exhibit,Farm
6,St-Sever Calvados,48.843608,-1.051698,6,6,Bank,French Restaurant,Zoo Exhibit,Fish & Chips Shop,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm


In [64]:
europe_merged = europe_merged.drop(columns = ['Cluster Labels'])
europe_merged.head()

,Train station,latitude,longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Digne-les-Bains,44.088710,6.222982,3,Park,Multiplex,French Restaurant,Restaurant,IT Services,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
2,Cagnes-sur-Mer,43.658130,7.148640,1,Supermarket,Bakery,Diner,Pharmacy,Falafel Restaurant,Plaza,Resort,Food Truck,Filipino Restaurant,Franconian Restaurant
3,Menton,43.774444,7.493300,2,Hotel,Italian Restaurant,Beach,Café,Mediterranean Restaurant,Bakery,Vegetarian / Vegan Restaurant,Pedestrian Plaza,Resort,Ice Cream Shop
5,Decazeville Centre,44.561434,2.255896,3,Supermarket,Medical Supply Store,Toy / Game Store,Pizza Place,Mobile Phone Shop,Miscellaneous Shop,Fast Food Restaurant,Photography Studio,Zoo Exhibit,Farm
6,St-Sever Calvados,48.843608,-1.051698,6,Bank,French Restaurant,Zoo Exhibit,Fish & Chips Shop,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm


## Visualizing clusters across Europe

In [69]:
europe_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, location, cluster in zip(europe_merged['latitude'], europe_merged['longitude'], europe_merged['Train station'], europe_merged['Cluster_Labels']):
    label = folium.Popup('{} - Cluster {}'.format(location, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(europe_clusters)
       
europe_clusters

## Examine clusters

### Cluster 0 (Highlighted by Red)

In [80]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 0, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Montmédy,Campground,Nature Preserve,Café,Art Gallery,Fast Food Restaurant,Financial or Legal Service,Film Studio,Filipino Restaurant,Field,Zoo Exhibit
137,Privas,French Restaurant,Café,Plaza,Ice Cream Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Financial or Legal Service
156,Luzech,Farm,Butcher,Residential Building (Apartment / Condo),Scenic Lookout,Café,Zoo Exhibit,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
190,Lusigny-sur-Barse Centre,Bank,Pizza Place,Zoo Exhibit,Financial or Legal Service,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant
232,Creutzwald Gare Routière,Furniture / Home Store,French Restaurant,Clothing Store,Café,Zoo Exhibit,Farmers Market,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
788,Uerikon Bahnhof,Bar,Gym / Fitness Center,Café,Scenic Lookout,History Museum,Zoo Exhibit,Falafel Restaurant,Fast Food Restaurant,Farmers Market,Farm
789,Bubikon Bahnhof,Restaurant,Flower Shop,Café,Historic Site,Gas Station,Zoo Exhibit,Farm,Field,Fast Food Restaurant,Farmers Market
790,Rüti ZH Bahnhof,Café,Electronics Store,Zoo Exhibit,Ethiopian Restaurant,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
815,Bari Centrale FNB,Café,Pizza Place,Italian Restaurant,Pub,Hotel,Ice Cream Shop,Clothing Store,Seafood Restaurant,Coffee Shop,Plaza


### Cluster 1 (Highlighted by Purple)

In [81]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 1, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Cagnes-sur-Mer,Supermarket,Bakery,Diner,Pharmacy,Falafel Restaurant,Plaza,Resort,Food Truck,Filipino Restaurant,Franconian Restaurant
7,Vire,Supermarket,Steakhouse,Automotive Shop,Fast Food Restaurant,Farm,Film Studio,Filipino Restaurant,Field,Farmers Market,Fair
9,Uzerche,Laundromat,Supermarket,Discount Store,Garden Center,Farmers Market,Financial or Legal Service,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant
18,Dole Ville,Plaza,Furniture / Home Store,Bar,Supermarket,French Restaurant,Fountain,Football Stadium,Event Space,Exhibit,Fabric Shop
24,Mayenne,Supermarket,Restaurant,Castle,Park,Historic Site,Flower Shop,Funeral Home,Plaza,Food Court,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
757,Düdingen Bahnhof,Supermarket,Grocery Store,Gas Station,Falafel Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Fair
767,Sarnen Bahnhof,Supermarket,Pizza Place,Café,Grocery Store,Lake,Gas Station,Chinese Restaurant,Field,Fast Food Restaurant,Farmers Market
771,Thayngen Bahnhof,Bowling Alley,Food & Drink Shop,Tunnel,Supermarket,Franconian Restaurant,Film Studio,Event Space,Exhibit,Fabric Shop,Factory
777,Uzwil Bahnhof,Supermarket,Hotel,Platform,Swiss Restaurant,Bakery,Shopping Mall,Gastropub,Farm,Filipino Restaurant,Field


### Cluster 2 (Highlighted by Blue)

In [82]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 2, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Menton,Hotel,Italian Restaurant,Beach,Café,Mediterranean Restaurant,Bakery,Vegetarian / Vegan Restaurant,Pedestrian Plaza,Resort,Ice Cream Shop
10,Dijon Ville,Hotel,French Restaurant,Plaza,Café,Pub,Bar,Tourist Information Center,Gourmet Shop,Gastropub,Garden
13,Colomiers,Japanese Restaurant,Gym Pool,Gas Station,Hotel,Exhibit,Fabric Shop,Factory,Fair,Financial or Legal Service,Event Space
17,Moirans,Organic Grocery,Arts & Crafts Store,Hotel,Zoo Exhibit,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
19,Rezé—Pont-Rousseau,Hotel,Tram Station,Boutique,Cultural Center,Fish Market,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
780,Urnäsch Bahnhof,Hotel,Supermarket,Ski Area,Swiss Restaurant,Plaza,Convenience Store,Construction & Landscaping,Farm,Filipino Restaurant,Field
783,Arbon Bahnhof,Shopping Mall,Hotel,Harbor / Marina,Park,Playground,Castle,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
785,Davos Platz Bahnhof,Hotel,Supermarket,Ski Area,Bar,Nightclub,Swiss Restaurant,Pizza Place,Diner,Steakhouse,Tapas Restaurant
813,Leukerbad Bahnhof,Hotel,Steakhouse,Swiss Restaurant,Restaurant,Water Park,Gym Pool,Market,Pizza Place,Light Rail Station,Hotel Pool


### Cluster 3 (Highlighted by navy blue)

In [83]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 3, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Digne-les-Bains,Park,Multiplex,French Restaurant,Restaurant,IT Services,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
5,Decazeville Centre,Supermarket,Medical Supply Store,Toy / Game Store,Pizza Place,Mobile Phone Shop,Miscellaneous Shop,Fast Food Restaurant,Photography Studio,Zoo Exhibit,Farm
12,Besançon Viotte,Hotel,French Restaurant,Bar,Italian Restaurant,Multiplex,Sushi Restaurant,Tea Room,Bed & Breakfast,Tourist Information Center,Photography Studio
14,Labège Innopole,French Restaurant,Asian Restaurant,Hotel,Italian Restaurant,Diner,Fast Food Restaurant,Men's Store,Perfume Shop,Shop & Service,Bakery
15,Albas,Italian Restaurant,Cricket Ground,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market
...,...,...,...,...,...,...,...,...,...,...,...
806,Kloten Bahnhof,Supermarket,Swiss Restaurant,Bus Station,Hotel,Italian Restaurant,Bar,Asian Restaurant,Athletics & Sports,Liquor Store,Bakery
807,Wallisellen Bahnhof,Supermarket,Italian Restaurant,Hotel,Café,Restaurant,Shopping Mall,Asian Restaurant,Light Rail Station,Perfume Shop,Bakery
808,Erlenbach ZH Bahnhof,Bakery,Italian Restaurant,Convenience Store,Pier,Fast Food Restaurant,Beach,Restaurant,Asian Restaurant,Grocery Store,Sports Club
810,Glattbrugg Bahnhof,Italian Restaurant,Supermarket,Kebab Restaurant,Swiss Restaurant,Tram Station,Hotel,Chinese Restaurant,Thai Restaurant,Japanese Restaurant,Bus Stop


### Cluster 4 (Highlighted by light blue)

In [84]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 4, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Bar-sur-Aube,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Hotel,Fabric Shop,Exhibit,Factory,Fair,Financial or Legal Service,Falafel Restaurant
250,Grenchen Nord,Grocery Store,Supermarket,Historic Site,Hotel,Bar,Football Stadium,Event Space,Exhibit,Fabric Shop,French Restaurant
372,New Mills Central,Grocery Store,Pub,Zoo Exhibit,Financial or Legal Service,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant
376,Prestwick Town,Bar,Grocery Store,Pub,Beach,Golf Course,Bistro,Ice Cream Shop,Newsagent,French Restaurant,Food Stand
381,Folkestone Central,Grocery Store,Supermarket,Soccer Field,Fish & Chips Shop,Gym / Fitness Center,Mexican Restaurant,Athletics & Sports,Fast Food Restaurant,Sandwich Place,Bar
383,Tilbury Town,Port,IT Services,Grocery Store,Flea Market,Zoo Exhibit,Farmers Market,Filipino Restaurant,Field,Fast Food Restaurant,Farm
402,Sudbury Hill Harrow,Construction & Landscaping,Gym / Fitness Center,Metro Station,Grocery Store,Gastropub,Soccer Field,Zoo Exhibit,Farm,Field,Fast Food Restaurant
475,Frick,Bakery,Gym / Fitness Center,Grocery Store,Supermarket,Fabric Shop,Exhibit,Factory,Event Space,Financial or Legal Service,Falafel Restaurant
476,Stein-Säckingen,Border Crossing,Bakery,Bistro,Gas Station,Convenience Store,Grocery Store,Falafel Restaurant,Farm,Fair,Fish Market
489,Sierre/Siders,Supermarket,Lake,Grocery Store,Modern European Restaurant,Hotel,French Restaurant,Football Stadium,Film Studio,Exhibit,Fabric Shop


### Cluster 5 (Highlighted by Biege)

In [85]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 5, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Marans,Rental Car Location,Construction & Landscaping,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Fair,Fish & Chips Shop,Falafel Restaurant
33,Nomain,Construction & Landscaping,Brewery,Farm,Financial or Legal Service,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Zoo Exhibit
35,Jeumont,Construction & Landscaping,Auto Workshop,Farm,Financial or Legal Service,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Zoo Exhibit
62,Lexy,Construction & Landscaping,Auto Garage,Farm,Financial or Legal Service,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Zoo Exhibit
69,Gudmont,Construction & Landscaping,Zoo Exhibit,Fish & Chips Shop,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
76,Fronville,Construction & Landscaping,Fast Food Restaurant,Zoo Exhibit,Fish & Chips Shop,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant
77,Eurville,Construction & Landscaping,Zoo Exhibit,Fish & Chips Shop,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
80,Kuntzig,Health & Beauty Service,Furniture / Home Store,Construction & Landscaping,Event Space,Exhibit,Ethiopian Restaurant,Fabric Shop,Factory,Financial or Legal Service,Fair
126,Torfou,Construction & Landscaping,Zoo Exhibit,Fish & Chips Shop,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
160,Capvern,Campground,Construction & Landscaping,Zoo Exhibit,Fish & Chips Shop,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm


### Cluster 6 (Highlighted by orange)

In [86]:
europe_merged.loc[europe_merged['Cluster_Labels'] == 6, europe_merged.columns[[0] + \
                                                                                 list(range(4, europe_merged.shape[1]))]]

,Train station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,St-Sever Calvados,Bank,French Restaurant,Zoo Exhibit,Fish & Chips Shop,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
94,Philippsbourg,French Restaurant,Zoo Exhibit,Fish & Chips Shop,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
112,Le Merlerault,Bank,Factory,Zoo Exhibit,Ethiopian Restaurant,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market
128,Pouzauges,French Restaurant,Zoo Exhibit,Fish & Chips Shop,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm
216,St-Amand-Longpré,Bank,Zoo Exhibit,Ethiopian Restaurant,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market
